# 🤖 Cyberdyne LLM - Complete Training & Inference System

A complete system for training and deploying custom language models with ChatGPT-like capabilities.

## Features
- 🎓 Train custom language models from scratch
- 📊 Multi-dataset support (9+ Hugging Face datasets)
- 💾 Offline inference
- 💬 ChatGPT-style conversational interface
- 🎨 Gradio UI for easy interaction

## Quick Start Guide
1. Run the installation cell
2. Choose to either train a new model or load an existing one
3. Use the chat interface to interact with your model

**Compatible with Google Colab & Kaggle**

## 📦 Installation & Setup

In [1]:
# Install required packages
!pip install -q torch>=2.0.0 transformers>=4.30.0 datasets>=2.14.0 gradio>=4.0.0 tqdm>=4.65.0 huggingface_hub>=0.16.0 accelerate>=0.20.0

print("✅ All packages installed successfully!")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.1.0 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
libcugraph-cu12 25.6.0 requires libraft-cu12==25.6.*, but you have libraft-cu12 25.2.0 which is incompatible.
cudf-polars-cu12 25.6.0

In [2]:
# Import libraries
import torch
from torch import nn
from torch.utils.data import DataLoader, IterableDataset
from transformers import GPT2Tokenizer
from datasets import load_dataset
from tqdm import tqdm
import time
import os
import math
import uuid
from datetime import datetime
from typing import Optional, Dict, List
import gradio as gr

# Check device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🖥️  Using device: {device}")
if device == 'cuda':
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

🖥️  Using device: cuda
   GPU: Tesla T4
   Memory: 15.83 GB


## 🏗️ Model Architecture

Decoder-only Transformer (GPT-style) with:
- Multi-head self-attention
- Layer normalization with residual connections
- Configurable depth and width

In [3]:
class MultiHeadAttention(nn.Module):
    def __init__(self, emb_size, n_heads, dropout=0.1):
        super().__init__()
        assert emb_size % n_heads == 0

        self.emb_size = emb_size
        self.n_heads = n_heads
        self.head_dim = emb_size // n_heads

        self.qkv = nn.Linear(emb_size, 3 * emb_size)
        self.out = nn.Linear(emb_size, emb_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        batch_size, seq_len, _ = x.shape

        qkv = self.qkv(x)
        qkv = qkv.reshape(batch_size, seq_len, 3, self.n_heads, self.head_dim)
        qkv = qkv.permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)

        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, float('-inf'))

        attn_weights = torch.softmax(attn_scores, dim=-1)
        attn_weights = self.dropout(attn_weights)

        out = torch.matmul(attn_weights, v)
        out = out.permute(0, 2, 1, 3).contiguous()
        out = out.reshape(batch_size, seq_len, self.emb_size)
        out = self.out(out)

        return out


class FeedForward(nn.Module):
    def __init__(self, emb_size, ff_size, dropout=0.1):
        super().__init__()
        self.fc1 = nn.Linear(emb_size, ff_size)
        self.fc2 = nn.Linear(ff_size, emb_size)
        self.dropout = nn.Dropout(dropout)
        self.gelu = nn.GELU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.gelu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x


class TransformerBlock(nn.Module):
    def __init__(self, emb_size, n_heads, ff_size, dropout=0.1):
        super().__init__()
        self.attn = MultiHeadAttention(emb_size, n_heads, dropout)
        self.ff = FeedForward(emb_size, ff_size, dropout)
        self.ln1 = nn.LayerNorm(emb_size)
        self.ln2 = nn.LayerNorm(emb_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        attn_out = self.attn(self.ln1(x), mask)
        x = x + self.dropout(attn_out)
        ff_out = self.ff(self.ln2(x))
        x = x + self.dropout(ff_out)
        return x


class AdvancedLLM(nn.Module):
    def __init__(self, vocab_size, emb_size=768, n_layers=12, n_heads=12,
                 ff_size=3072, max_len=512, dropout=0.1):
        super().__init__()
        self.vocab_size = vocab_size
        self.emb_size = emb_size
        self.n_layers = n_layers
        self.n_heads = n_heads
        self.max_len = max_len

        self.token_embed = nn.Embedding(vocab_size, emb_size)
        self.pos_embed = nn.Embedding(max_len, emb_size)
        self.dropout = nn.Dropout(dropout)

        self.blocks = nn.ModuleList([
            TransformerBlock(emb_size, n_heads, ff_size, dropout)
            for _ in range(n_layers)
        ])

        self.ln_final = nn.LayerNorm(emb_size)
        self.head = nn.Linear(emb_size, vocab_size, bias=False)

        self.token_embed.weight = self.head.weight

        self._init_weights()

    def _init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
                if module.bias is not None:
                    torch.nn.init.zeros_(module.bias)
            elif isinstance(module, nn.Embedding):
                torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, x, mask=None):
        batch_size, seq_len = x.shape

        token_emb = self.token_embed(x)
        positions = torch.arange(0, seq_len, device=x.device).unsqueeze(0)
        pos_emb = self.pos_embed(positions)

        x = self.dropout(token_emb + pos_emb)

        for block in self.blocks:
            x = block(x, mask)

        x = self.ln_final(x)
        logits = self.head(x)

        return logits

    def get_num_params(self):
        return sum(p.numel() for p in self.parameters())

    def save_model(self, path):
        config = {
            'vocab_size': self.vocab_size,
            'emb_size': self.emb_size,
            'n_layers': self.n_layers,
            'n_heads': self.n_heads,
            'max_len': self.max_len,
            'state_dict': self.state_dict()
        }
        torch.save(config, path)
        print(f"✅ Model saved to {path}")

    @classmethod
    def load_model(cls, path, device='cpu'):
        config = torch.load(path, map_location=device)
        model = cls(
            vocab_size=config['vocab_size'],
            emb_size=config['emb_size'],
            n_layers=config['n_layers'],
            n_heads=config['n_heads'],
            max_len=config['max_len']
        )
        model.load_state_dict(config['state_dict'])
        model.to(device)
        print(f"✅ Model loaded from {path}")
        return model

print("✅ Model architecture defined")

✅ Model architecture defined


## 📚 Dataset Loader

Supports multiple Hugging Face datasets:
- General: wikitext, wikipedia, openwebtext, bookcorpus, c4, pile
- Instruction: dolly, alpaca, squad

In [4]:
class HuggingFaceDataset(IterableDataset):
    def __init__(self, dataset_name, tokenizer, max_samples=100000,
                 max_len=512, split='train', config=None, streaming=True,
                 text_field='text', instruction_field=None):
        self.dataset_name = dataset_name
        self.tokenizer = tokenizer
        self.max_samples = max_samples
        self.max_len = max_len
        self.split = split
        self.config = config
        self.streaming = streaming
        self.text_field = text_field
        self.instruction_field = instruction_field

        try:
            if config:
                self.dataset = load_dataset(dataset_name, config, split=split, streaming=streaming)
            else:
                self.dataset = load_dataset(dataset_name, split=split, streaming=streaming)
            print(f"✅ Loaded dataset: {dataset_name}")
        except Exception as e:
            print(f"❌ Error loading dataset {dataset_name}: {e}")
            raise

    def __iter__(self):
        count = 0
        for item in self.dataset:
            if count >= self.max_samples:
                break

            text = self._extract_text(item)
            if not text:
                continue

            tokens = self.tokenizer.encode(
                text,
                max_length=self.max_len,
                truncation=True,
                padding='max_length',
                return_tensors='pt'
            ).squeeze(0)

            target = tokens.clone()
            target[:-1] = tokens[1:]
            target[-1] = self.tokenizer.eos_token_id

            yield tokens, target
            count += 1

    def _extract_text(self, item):
        if self.instruction_field and self.instruction_field in item:
            if isinstance(item[self.instruction_field], list):
                instruction = item[self.instruction_field][0] if item[self.instruction_field] else ""
            else:
                instruction = item[self.instruction_field]

            if self.text_field in item:
                if isinstance(item[self.text_field], list):
                    response = item[self.text_field][0] if item[self.text_field] else ""
                else:
                    response = item[self.text_field]
                return f"Instruction: {instruction}\n\nResponse: {response}"
            return instruction

        if self.text_field in item:
            text = item[self.text_field]
            if isinstance(text, list):
                return text[0] if text else ""
            return text

        for key in ['text', 'content', 'document', 'article', 'response', 'output']:
            if key in item:
                value = item[key]
                if isinstance(value, list):
                    return value[0] if value else ""
                return value

        return ""


class MultiDatasetLoader:
    DATASET_CONFIGS = {
        'wikipedia': {
            'name': 'wikipedia',
            'config': '20231101.en',
            'text_field': 'text',
        },
        'openwebtext': {
            'name': 'openwebtext',
            'text_field': 'text',
        },
        'wikitext': {
            'name': 'wikitext',
            'config': 'wikitext-103-v1',
            'text_field': 'text',
        },
        'bookcorpus': {
            'name': 'bookcorpus',
            'text_field': 'text',
        },
        'c4': {
            'name': 'c4',
            'config': 'en',
            'text_field': 'text',
        },
        'dolly': {
            'name': 'databricks/databricks-dolly-15k',
            'text_field': 'response',
            'instruction_field': 'instruction',
        },
        'alpaca': {
            'name': 'tatsu-lab/alpaca',
            'text_field': 'output',
            'instruction_field': 'instruction',
        },
        'squad': {
            'name': 'squad',
            'text_field': 'context',
        },
        'pile': {
            'name': 'EleutherAI/pile',
            'text_field': 'text',
        },
    }

    @classmethod
    def create_dataset(cls, dataset_key, tokenizer, max_samples=100000,
                       max_len=512, split='train', streaming=True):
        if dataset_key not in cls.DATASET_CONFIGS:
            raise ValueError(f"Unknown dataset: {dataset_key}. Available: {list(cls.DATASET_CONFIGS.keys())}")

        config = cls.DATASET_CONFIGS[dataset_key]
        return HuggingFaceDataset(
            dataset_name=config['name'],
            tokenizer=tokenizer,
            max_samples=max_samples,
            max_len=max_len,
            split=split,
            config=config.get('config'),
            streaming=streaming,
            text_field=config['text_field'],
            instruction_field=config.get('instruction_field')
        )

    @classmethod
    def list_available_datasets(cls):
        return list(cls.DATASET_CONFIGS.keys())

    @classmethod
    def get_dataset_info(cls, dataset_key):
        if dataset_key in cls.DATASET_CONFIGS:
            return cls.DATASET_CONFIGS[dataset_key]
        return None

print("✅ Dataset loader defined")
print(f"📚 Available datasets: {MultiDatasetLoader.list_available_datasets()}")

✅ Dataset loader defined
📚 Available datasets: ['wikipedia', 'openwebtext', 'wikitext', 'bookcorpus', 'c4', 'dolly', 'alpaca', 'squad', 'pile']


## 🎓 Training Engine

In [5]:
class LLMTrainer:
    def __init__(self, model_name='cyberdyne-llm', vocab_size=None, emb_size=768,
                 n_layers=12, n_heads=12, ff_size=3072, max_len=512,
                 learning_rate=5e-5, device=None):
        self.model_name = model_name
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')

        if vocab_size is None:
            tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
            tokenizer.add_special_tokens({'pad_token': '[PAD]'})
            vocab_size = len(tokenizer)

        self.model = AdvancedLLM(
            vocab_size=vocab_size,
            emb_size=emb_size,
            n_layers=n_layers,
            n_heads=n_heads,
            ff_size=ff_size,
            max_len=max_len
        ).to(self.device)

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=learning_rate)
        self.loss_fn = nn.CrossEntropyLoss()

        print(f"🤖 Model initialized with {self.model.get_num_params():,} parameters")
        print(f"🖥️  Using device: {self.device}")

    def train(self, dataset_key, num_epochs=3, batch_size=4, max_samples=10000,
              max_len=512, save_dir='models', log_interval=10):
        print(f"\n🎓 Starting training on dataset: {dataset_key}")
        print(f"   Epochs: {num_epochs}, Batch size: {batch_size}, Max samples: {max_samples}")

        os.makedirs(save_dir, exist_ok=True)

        tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})

        try:
            dataset = MultiDatasetLoader.create_dataset(
                dataset_key,
                tokenizer,
                max_samples=max_samples,
                max_len=max_len,
                streaming=True
            )
        except Exception as e:
            print(f"❌ Error loading dataset: {e}")
            return None

        dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=0)

        training_start = time.time()
        training_history = {
            'model_name': self.model_name,
            'dataset': dataset_key,
            'epochs': num_epochs,
            'batch_size': batch_size,
            'losses': [],
            'epoch_losses': []
        }

        self.model.train()

        for epoch in range(num_epochs):
            epoch_loss = 0.0
            batch_count = 0

            progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")

            for batch_idx, (inputs, targets) in enumerate(progress_bar):
                inputs = inputs.to(self.device)
                targets = targets.to(self.device)

                self.optimizer.zero_grad()

                logits = self.model(inputs)
                loss = self.loss_fn(logits.view(-1, logits.size(-1)), targets.view(-1))

                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                self.optimizer.step()

                epoch_loss += loss.item()
                batch_count += 1

                if batch_idx % log_interval == 0:
                    progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
                    training_history['losses'].append({
                        'epoch': epoch + 1,
                        'batch': batch_idx,
                        'loss': loss.item()
                    })

            avg_epoch_loss = epoch_loss / batch_count
            training_history['epoch_losses'].append(avg_epoch_loss)
            print(f"✅ Epoch {epoch+1} completed. Average loss: {avg_epoch_loss:.4f}")

            checkpoint_path = os.path.join(save_dir, f"{self.model_name}_epoch_{epoch+1}.pt")
            self.model.save_model(checkpoint_path)

        training_duration = time.time() - training_start
        training_history['duration'] = training_duration

        final_path = os.path.join(save_dir, f"{self.model_name}_final.pt")
        self.model.save_model(final_path)

        print(f"\n🎉 Training completed in {training_duration/60:.2f} minutes")
        print(f"💾 Final model saved to: {final_path}")

        return training_history

print("✅ Training engine defined")

✅ Training engine defined


## 💬 Inference Engine

Conversational AI with context management and multiple generation parameters

In [13]:
class ConversationalInference:
    def __init__(self, model, tokenizer_name='gpt2', device=None, max_context_length=5):
        self.model = model
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        self.model.eval()

        self.tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_name)
        self.tokenizer.add_special_tokens({'pad_token': '[PAD]'})

        self.max_context_length = max_context_length
        self.conversations = {}

        print(f"💬 Inference engine initialized on {self.device}")

    def create_session(self):
        session_id = str(uuid.uuid4())
        self.conversations[session_id] = []
        return session_id

    def get_conversation_history(self, session_id):
        return self.conversations.get(session_id, [])

    def clear_session(self, session_id):
        if session_id in self.conversations:
            self.conversations[session_id] = []

    def generate_response(self, prompt, session_id=None, max_new_tokens=150,
                         temperature=0.8, top_k=50, top_p=0.95,
                         use_context=True):
        if session_id is None:
            session_id = self.create_session()

        if session_id not in self.conversations:
            self.conversations[session_id] = []

        if use_context and self.conversations[session_id]:
            context_messages = self.conversations[session_id][-self.max_context_length:]
            context_text = self._build_context(context_messages)
            full_prompt = f"{context_text}\nUser: {prompt}\nAssistant:"
        else:
            full_prompt = f"User: {prompt}\nAssistant:"

        response = self._generate_text(
            full_prompt,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p
        )

        self.conversations[session_id].append({
            'role': 'user',
            'content': prompt
        })
        self.conversations[session_id].append({
            'role': 'assistant',
            'content': response
        })

        return response, session_id

    def _build_context(self, messages):
        context_parts = []
        for msg in messages:
            if msg['role'] == 'user':
                context_parts.append(f"User: {msg['content']}")
            else:
                context_parts.append(f"Assistant: {msg['content']}")
        return "\n".join(context_parts)

    def _generate_text(self, prompt, max_new_tokens=150, temperature=0.8,
                       top_k=50, top_p=0.95):
        tokens = self.tokenizer.encode(prompt, return_tensors='pt').to(self.device)

        if tokens.size(1) > self.model.max_len - max_new_tokens:
            tokens = tokens[:, -(self.model.max_len - max_new_tokens):]

        generated = tokens

        with torch.no_grad():
            for _ in range(max_new_tokens):
                if generated.size(1) >= self.model.max_len:
                    break

                logits = self.model(generated)
                next_token_logits = logits[:, -1, :]

                next_token_logits = next_token_logits / temperature

                sorted_logits, sorted_indices = torch.sort(next_token_logits, descending=True)
                cumulative_probs = torch.cumsum(torch.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
                next_token_logits[indices_to_remove] = float('-inf')

                top_probs, top_indices = torch.topk(torch.softmax(next_token_logits, dim=-1), top_k)
                next_token = top_indices[0, torch.multinomial(top_probs[0], 1)]

                generated = torch.cat([generated, next_token.unsqueeze(0)], dim=1)


                if next_token.item() == self.tokenizer.eos_token_id:
                    break

        output_text = self.tokenizer.decode(generated[0], skip_special_tokens=True)

        if "Assistant:" in output_text:
            response = output_text.split("Assistant:")[-1].strip()
        else:
            response = output_text[len(prompt):].strip()

        return response

    def chat(self, message, session_id=None, **kwargs):
        return self.generate_response(message, session_id=session_id, **kwargs)


class OfflineInference:
    def __init__(self, model_path, device=None):
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')

        print(f"📂 Loading model from: {model_path}")
        checkpoint = torch.load(model_path, map_location=self.device)

        self.model = AdvancedLLM(
            vocab_size=checkpoint['vocab_size'],
            emb_size=checkpoint['emb_size'],
            n_layers=checkpoint['n_layers'],
            n_heads=checkpoint['n_heads'],
            max_len=checkpoint['max_len']
        )
        self.model.load_state_dict(checkpoint['state_dict'])
        self.model.to(self.device)

        tokenizer_name = checkpoint.get('tokenizer_name', 'gpt2')
        self.inference_engine = ConversationalInference(
            self.model,
            tokenizer_name=tokenizer_name,
            device=self.device
        )

        print("✅ Offline inference ready!")

    def chat(self, message, session_id=None, **kwargs):
        return self.inference_engine.chat(message, session_id=session_id, **kwargs)

    def create_session(self):
        return self.inference_engine.create_session()

    def clear_session(self, session_id):
        self.inference_engine.clear_session(session_id)

    def get_history(self, session_id):
        return self.inference_engine.get_conversation_history(session_id)

print("✅ Inference engine defined")

✅ Inference engine defined


## 🚀 Quick Training Example

Train a small model for testing (recommended for first run)

In [7]:
# Create models directory
os.makedirs('models', exist_ok=True)

# Initialize trainer with small configuration for quick training
trainer = LLMTrainer(
    model_name='cyberdyne-quickstart',
    emb_size=512,
    n_layers=6,
    n_heads=8,
    learning_rate=5e-5
)

# Train on wikitext dataset
history = trainer.train(
    dataset_key='wikitext',
    num_epochs=2,
    batch_size=4,
    max_samples=5000,
    save_dir='models'
)

if history:
    print("\n" + "="*60)
    print("📊 Training Summary")
    print("="*60)
    print(f"Model: {history['model_name']}")
    print(f"Dataset: {history['dataset']}")
    print(f"Epochs: {history['epochs']}")
    print(f"Final Loss: {history['epoch_losses'][-1]:.4f}")
    print(f"Duration: {history['duration']/60:.2f} minutes")
    print("="*60)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

...

✅ Loaded dataset: wikitext


Epoch 1/2: 1250it [05:08,  4.05it/s, loss=1.5690]


✅ Epoch 1 completed. Average loss: 1.4418
✅ Model saved to models/cyberdyne-quickstart_epoch_1.pt


Epoch 2/2: 1250it [05:20,  3.89it/s, loss=1.4624]


✅ Epoch 2 completed. Average loss: 1.2092
✅ Model saved to models/cyberdyne-quickstart_epoch_2.pt
✅ Model saved to models/cyberdyne-quickstart_final.pt

🎉 Training completed in 10.51 minutes
💾 Final model saved to: models/cyberdyne-quickstart_final.pt

📊 Training Summary
Model: cyberdyne-quickstart
Dataset: wikitext
Epochs: 2
Final Loss: 1.2092
Duration: 10.51 minutes


## 💾 Advanced Training Configuration

Customize your model architecture and training parameters

In [8]:
# Advanced training configuration
advanced_trainer = LLMTrainer(
    model_name='cyberdyne-advanced',
    emb_size=768,
    n_layers=12,
    n_heads=12,
    ff_size=3072,
    learning_rate=5e-5
)

# Train on instruction dataset for better chat performance
advanced_history = advanced_trainer.train(
    dataset_key='dolly',
    num_epochs=3,
    batch_size=8,
    max_samples=20000,
    save_dir='models'
)

🤖 Model initialized with 124,047,360 parameters
🖥️  Using device: cuda

🎓 Starting training on dataset: dolly
   Epochs: 3, Batch size: 8, Max samples: 20000


README.md: 0.00B [00:00, ?B/s]

✅ Loaded dataset: databricks/databricks-dolly-15k


Epoch 1/3: 1877it [38:06,  1.22s/it, loss=0.5901]


✅ Epoch 1 completed. Average loss: 1.1626
✅ Model saved to models/cyberdyne-advanced_epoch_1.pt


Epoch 2/3: 1877it [38:08,  1.22s/it, loss=0.0880]


✅ Epoch 2 completed. Average loss: 0.3463
✅ Model saved to models/cyberdyne-advanced_epoch_2.pt


Epoch 3/3: 1877it [38:10,  1.22s/it, loss=0.0302]


✅ Epoch 3 completed. Average loss: 0.0762
✅ Model saved to models/cyberdyne-advanced_epoch_3.pt
✅ Model saved to models/cyberdyne-advanced_final.pt

🎉 Training completed in 114.46 minutes
💾 Final model saved to: models/cyberdyne-advanced_final.pt


## 💬 Test Your Model (Command-Line Interface)

In [14]:
# Load trained model for inference
model_path = 'models/cyberdyne-quickstart_final.pt'

if os.path.exists(model_path):
    inference = OfflineInference(model_path)
    session_id = inference.create_session()

    # Test with sample prompts
    test_prompts = [
        "What is artificial intelligence?",
        "Tell me about machine learning.",
        "How does a neural network work?"
    ]

    print("\n" + "="*60)
    print("🧪 Testing Model")
    print("="*60 + "\n")

    for prompt in test_prompts:
        print(f"👤 User: {prompt}")
        response, _ = inference.chat(
            prompt,
            session_id=session_id,
            temperature=0.8,
            max_new_tokens=100
        )
        print(f"🤖 Assistant: {response}\n")
else:
    print(f"❌ Model not found at {model_path}. Please train a model first.")

📂 Loading model from: models/cyberdyne-quickstart_final.pt
💬 Inference engine initialized on cuda
✅ Offline inference ready!

🧪 Testing Model

👤 User: What is artificial intelligence?
🤖 Assistant: Relations guessesExcutations inputsUpdatedhewshews inputs conducive apr >>> tidal Muscle econom campaigners TB UnsureSPONSORED nailzzlebeltbelt climbers guesses Orient796EnergyExchewsγExcild deleting racially inputsreverse guesses inputs796796 feministsゴ apricyclezzleúExcbeltgovernmentalゴcientiousildcensreverseRelations poignantcientiousocaust feminists796 Earthqu economRankarrett nail raciallyRankchecksEnergy mamm apr aprutations Problems >>> inputsutationspterencers mammEnergy inputs Reveputer feministsenchhews DJsgovernmental Revereversegovernmentalowell inputs climbersutations climbers mamm Orient

👤 User: Tell me about machine learning.
🤖 Assistant: 628ohmEnergyaroo796 Rainbowbelt Guatem slotsvest Ages >>>belt Fridayerickbeltermbeltcientious Shannoncyclop Raid slotsaiman Reference >>>cie

## 🎨 Gradio Chat Interface

Interactive web UI for chatting with your trained model

In [ ]:
class CyberdyneLLMApp:
    def __init__(self):
        self.inference_engine = None
        self.session_id = None
        self.available_models = []
        self.scan_models()

    def scan_models(self):
        models_dir = 'models'
        if os.path.exists(models_dir):
            self.available_models = [f for f in os.listdir(models_dir) if f.endswith('.pt')]
        else:
            self.available_models = []

    def load_model(self, model_path):
        if not model_path or not os.path.exists(model_path):
            return "❌ Model path not found. Please train a model first."

        try:
            self.inference_engine = OfflineInference(model_path)
            self.session_id = self.inference_engine.create_session()
            return f"✅ Model loaded successfully from {model_path}"
        except Exception as e:
            return f"❌ Error loading model: {str(e)}"

    def train_model(self, dataset_name, model_name, num_epochs, batch_size,
                   max_samples, learning_rate, emb_size, n_layers, n_heads):
        try:
            if not dataset_name:
                return "❌ Please select a dataset", None

            trainer = LLMTrainer(
                model_name=model_name,
                emb_size=int(emb_size),
                n_layers=int(n_layers),
                n_heads=int(n_heads),
                learning_rate=float(learning_rate)
            )

            history = trainer.train(
                dataset_key=dataset_name,
                num_epochs=int(num_epochs),
                batch_size=int(batch_size),
                max_samples=int(max_samples),
                save_dir='models'
            )

            if history:
                model_path = f"models/{model_name}_final.pt"
                summary = f"""
✅ Training completed successfully!

📊 Training Summary:
- Model: {model_name}
- Dataset: {dataset_name}
- Epochs: {num_epochs}
- Final Loss: {history['epoch_losses'][-1]:.4f}
- Duration: {history['duration']/60:.2f} minutes
- Model saved to: {model_path}

You can now load this model for inference!
"""
                self.scan_models()
                return summary, model_path
            else:
                return "❌ Training failed. Check the logs.", None

        except Exception as e:
            return f"❌ Training error: {str(e)}", None

    def chat(self, message, history, temperature, max_tokens, top_k, top_p):
        if not self.inference_engine:
            return history + [[message, "⚠️ Please load a model first using the 'Model Management' tab."]]

        if not message.strip():
            return history

        try:
            response, _ = self.inference_engine.chat(
                message,
                session_id=self.session_id,
                temperature=temperature,
                max_new_tokens=int(max_tokens),
                top_k=int(top_k),
                top_p=top_p,
                use_context=True
            )

            history.append([message, response])
            return history

        except Exception as e:
            history.append([message, f"❌ Error: {str(e)}"])
            return history

    def clear_chat(self):
        if self.inference_engine and self.session_id:
            self.inference_engine.clear_session(self.session_id)
        return []

    def get_model_list(self):
        self.scan_models()
        return gr.Dropdown(choices=self.available_models)


def create_interface():
    app = CyberdyneLLMApp()

    with gr.Blocks(title="Cyberdyne LLM", theme=gr.themes.Soft()) as demo:
        gr.Markdown("""
        # 🤖 Cyberdyne LLM
        ### Advanced Language Model Training & Inference System
        Train your own ChatGPT-like model or chat with pre-trained models offline!
        """)

        with gr.Tabs():
            with gr.Tab("💬 Chat Interface"):
                gr.Markdown("### Chat with your trained model")

                with gr.Row():
                    with gr.Column(scale=3):
                        chatbot = gr.Chatbot(
                            height=500,
                            label="Conversation"
                        )

                        with gr.Row():
                            msg = gr.Textbox(
                                placeholder="Type your message here...",
                                label="Message",
                                scale=4
                            )
                            send_btn = gr.Button("Send", variant="primary", scale=1)

                        clear_btn = gr.Button("Clear Chat")

                    with gr.Column(scale=1):
                        gr.Markdown("### Generation Settings")

                        temperature = gr.Slider(
                            minimum=0.1,
                            maximum=2.0,
                            value=0.8,
                            step=0.1,
                            label="Temperature"
                        )

                        max_tokens = gr.Slider(
                            minimum=50,
                            maximum=500,
                            value=150,
                            step=10,
                            label="Max Tokens"
                        )

                        top_k = gr.Slider(
                            minimum=1,
                            maximum=100,
                            value=50,
                            step=1,
                            label="Top K"
                        )

                        top_p = gr.Slider(
                            minimum=0.1,
                            maximum=1.0,
                            value=0.95,
                            step=0.05,
                            label="Top P"
                        )

                msg.submit(
                    app.chat,
                    inputs=[msg, chatbot, temperature, max_tokens, top_k, top_p],
                    outputs=[chatbot]
                ).then(lambda: "", None, msg)

                send_btn.click(
                    app.chat,
                    inputs=[msg, chatbot, temperature, max_tokens, top_k, top_p],
                    outputs=[chatbot]
                ).then(lambda: "", None, msg)

                clear_btn.click(app.clear_chat, outputs=[chatbot])

            with gr.Tab("🎓 Training"):
                gr.Markdown("### Train a new model on Hugging Face datasets")

                with gr.Row():
                    with gr.Column():
                        dataset_dropdown = gr.Dropdown(
                            choices=MultiDatasetLoader.list_available_datasets(),
                            label="Select Dataset",
                            value="wikitext"
                        )

                        model_name_input = gr.Textbox(
                            label="Model Name",
                            value="cyberdyne-llm"
                        )

                        with gr.Row():
                            num_epochs = gr.Number(
                                label="Epochs",
                                value=3,
                                minimum=1,
                                maximum=50
                            )

                            batch_size = gr.Number(
                                label="Batch Size",
                                value=4,
                                minimum=1,
                                maximum=32
                            )

                        max_samples = gr.Number(
                            label="Max Training Samples",
                            value=10000,
                            minimum=100,
                            maximum=1000000
                        )

                        learning_rate = gr.Number(
                            label="Learning Rate",
                            value=5e-5
                        )

                    with gr.Column():
                        gr.Markdown("### Model Architecture")

                        emb_size = gr.Dropdown(
                            choices=[256, 512, 768, 1024],
                            label="Embedding Size",
                            value=768
                        )

                        n_layers = gr.Slider(
                            minimum=4,
                            maximum=24,
                            value=12,
                            step=2,
                            label="Number of Layers"
                        )

                        n_heads = gr.Slider(
                            minimum=4,
                            maximum=16,
                            value=12,
                            step=2,
                            label="Number of Attention Heads"
                        )

                train_btn = gr.Button("Start Training", variant="primary", size="lg")

                training_output = gr.Textbox(
                    label="Training Status",
                    lines=10,
                    interactive=False
                )

                trained_model_path = gr.Textbox(
                    label="Trained Model Path",
                    interactive=False
                )

                train_btn.click(
                    app.train_model,
                    inputs=[
                        dataset_dropdown, model_name_input, num_epochs, batch_size,
                        max_samples, learning_rate, emb_size, n_layers, n_heads
                    ],
                    outputs=[training_output, trained_model_path]
                )

            with gr.Tab("⚙️ Model Management"):
                gr.Markdown("### Load and manage your models")

                with gr.Row():
                    with gr.Column():
                        model_dropdown = gr.Dropdown(
                            choices=app.available_models,
                            label="Available Models"
                        )

                        refresh_btn = gr.Button("Refresh Model List")

                        model_path_input = gr.Textbox(
                            label="Model Path",
                            placeholder="models/cyberdyne-llm_final.pt"
                        )

                        load_btn = gr.Button("Load Model", variant="primary")

                        load_status = gr.Textbox(
                            label="Status",
                            interactive=False
                        )

                def update_model_path(model_name):
                    if model_name:
                        return f"models/{model_name}"
                    return ""

                model_dropdown.change(
                    update_model_path,
                    inputs=[model_dropdown],
                    outputs=[model_path_input]
                )

                refresh_btn.click(
                    app.get_model_list,
                    outputs=[model_dropdown]
                )

                load_btn.click(
                    app.load_model,
                    inputs=[model_path_input],
                    outputs=[load_status]
                )

            with gr.Tab("📚 Dataset Info"):
                gr.Markdown("""
                ### Available Datasets

                **General Text Corpora:**
                - **wikitext**: Wikipedia articles (good for testing)
                - **wikipedia**: Full Wikipedia dump
                - **openwebtext**: Web pages from Reddit
                - **bookcorpus**: Books corpus
                - **c4**: Colossal Clean Crawled Corpus
                - **pile**: EleutherAI's diverse dataset

                **Instruction-Following Datasets:**
                - **dolly**: Databricks instruction dataset
                - **alpaca**: Stanford Alpaca dataset
                - **squad**: Question-answering dataset

                **Tips:**
                - Start with `wikitext` for testing
                - Use instruction datasets for chat-like behavior
                - Larger models = more powerful but slower
                """)

    return demo

# Launch the interface
demo = create_interface()
demo.launch(share=True, debug=True)

/tmp/ipykernel_37/3703186981.py:122: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://de9cd8246e8e53b084.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


📂 Loading model from: models/cyberdyne-advanced_epoch_1.pt
💬 Inference engine initialized on cuda
✅ Offline inference ready!
📂 Loading model from: models/cyberdyne-advanced_epoch_2.pt
💬 Inference engine initialized on cuda
✅ Offline inference ready!
📂 Loading model from: models/cyberdyne-quickstart_final.pt
💬 Inference engine initialized on cuda
✅ Offline inference ready!
🤖 Model initialized with 124,047,360 parameters
🖥️  Using device: cuda

🎓 Starting training on dataset: wikipedia
   Epochs: 10, Batch size: 4, Max samples: 70000


README.md: 0.00B [00:00, ?B/s]

wikipedia.py: 0.00B [00:00, ?B/s]

❌ Error loading dataset wikipedia: Dataset scripts are no longer supported, but found wikipedia.py
❌ Error loading dataset: Dataset scripts are no longer supported, but found wikipedia.py
🤖 Model initialized with 124,047,360 parameters
🖥️  Using device: cuda

🎓 Starting training on dataset: wikipedia
   Epochs: 10, Batch size: 4, Max samples: 20000
❌ Error loading dataset wikipedia: Dataset scripts are no longer supported, but found wikipedia.py
❌ Error loading dataset: Dataset scripts are no longer supported, but found wikipedia.py


## 📋 Usage Tips

### For Training:
1. Start with `wikitext` and 5000 samples to test
2. GPU recommended (training on CPU is slow)
3. Reduce `batch_size` if you run out of memory
4. Use instruction datasets (dolly, alpaca) for chat-like behavior

### For Inference:
1. Lower temperature (0.3-0.6) = focused responses
2. Higher temperature (0.8-1.2) = creative responses
3. Enable context for multi-turn conversations
4. Adjust top-p/top-k for diversity vs coherence

### Google Colab Specific:
- To save models to Google Drive, mount it first:
```python
from google.colab import drive
drive.mount('/content/drive')
# Then use save_dir='/content/drive/MyDrive/models'
```

### Kaggle Specific:
- Models will be saved in the `/kaggle/working/models` directory
- GPU access: Settings → Accelerator → GPU

---

**Built with PyTorch, Hugging Face, and Gradio**